#  Workarounds for the problem using grid equivalents
This tutorial gives some suggestions to solve the difficulties that arise when using grid equivalent.
In this tutorial, we still use the grid case9 as the test bed:

- internal area (buses): [0, 3, 4, 8]
- boundary buses: [4, 8] (boundary buses belong to the internal area)
- external area (buses): [1, 2, 5, 6, 7] 

<img src="pics/grid areas.png" alt="ALT">

In [ ]:
from pandapower.networks.power_system_test_cases import case9
from pandapower.grid_equivalents import get_equivalent
from pandapower.create import create_dcline
from pandapower.run import runpp
from copy import deepcopy

net = case9()

## A) dc line

If the to-be-reduced grid has dc-lines. We need convert them before starting grid equivalent. First of all, let's create a dc-line. In the following, the line (index=2) between bus 4 and bus 8 is replaced by a dc-line

In [ ]:
net.line.drop([2],inplace=True)
create_dcline(net, 4, 5, 50, 0.5, 0.7, 1.0, 1.0)
runpp(net)

Now, we have got a grid with dc-line (see A)).
<img src="pics/grid areas (workarounds).png" alt="ALT">

using *_add_dcline_gens* the dc-lines are represented by generators. Then, we need to remote or disconnect the origin dc_line or close them. 

In [ ]:
from pandapower.auxiliary import _add_dcline_gens
net2 = deepcopy(net)
_add_dcline_gens(net2)
net2.dcline.in_service=False
runpp(net2)

We can compare the power flow results between *net1* and *net2* and carry out the grid equivalent process.

In [ ]:
print("net with dcline", net.res_bus)
print("net with represented dcline", net2.res_bus)
net_eq = get_equivalent(net2, "rei", [4,8], [0])
print("equivalent net", net_eq.res_bus)

## B) slack in the external area

In some cases, there are slack buses in the external area, see B). It could cause power flow problems during the grid equivalent calculation. In this case, we can consider the slack buses as the boundary to reserve them.
In the grid *case9*, there are two generators in the external area. We activate their slack function and treat the corresponding buses as the boundary.

In [ ]:
net = case9()
net.gen.slack = True
runpp(net)
boundary_buses = [4, 8, 1, 2]
net_eq = get_equivalent(net, "rei", [4,8], [0])
print("original net", net.res_bus)
print("equivalent net", net_eq.res_bus)